In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=513b527eb8dcdb5cee401095826a16fcd4491549241ddb657c180acf9b075661
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import os
import wget
import tarfile

In [ ]:
from shutil import rmtree

In [ ]:
DATASET_URL = 'http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz'
ARCHIVE = os.path.basename(DATASET_URL)

In [ ]:
wget.download(DATASET_URL)

'speech_commands_v0.01.tar.gz'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'speech/'
import sys
sys.path.append(base_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from pathlib import Path
current_dir = Path.cwd()
data_dir = Path(current_dir,'data')
train_dir = Path(data_dir, 'train')
if os.path.exists(train_dir):
  rmtree(train_dir)
os.makedirs(train_dir)


In [ ]:
with tarfile.open(ARCHIVE, 'r:gz') as tar:
  # tar.extractall(path='data/train')
  tar.extractall(path=train_dir)

os.remove(ARCHIVE)

In [ ]:
from pathlib import Path
current_dir = Path.cwd()
data_dir = Path(current_dir,'data')
train_dir = Path(data_dir, 'train')
checkpt_dir = Path(current_dir,'checkpoint')
print("train_dir = ",train_dir)
print("checkpt_dir = ",checkpt_dir)

train_dir =  /content/data/train
checkpt_dir =  /content/checkpoint


In [ ]:
!pip install keras-applications==1.0.7
!pip install keras_preprocessing==1.0.1

     |████████████████████████████████| 61kB 4.8MB/s 
ERROR: tensorflow 2.3.0 has requirement keras-preprocessing<1.2,>=1.1.1, but you'll have keras-preprocessing 1.0.1 which is incompatible.
  Found existing installation: Keras-Preprocessing 1.1.2
    Uninstalling Keras-Preprocessing-1.1.2:
      Successfully uninstalled Keras-Preprocessing-1.1.2


In [ ]:
!pip install tensorflow
!pip install keras

     |████████████████████████████████| 51kB 3.6MB/s 
  Found existing installation: Keras-Preprocessing 1.0.1
    Uninstalling Keras-Preprocessing-1.0.1:
      Successfully uninstalled Keras-Preprocessing-1.0.1


In [ ]:
!pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to /tmp/pip-req-build-ls7ly9mr
  Running command git clone -q https://github.com/hyperopt/hyperopt-sklearn.git /tmp/pip-req-build-ls7ly9mr
     |████████████████████████████████| 163kB 9.3MB/s 
  Created wheel for hpsklearn: filename=hpsklearn-0.0.3-cp36-none-any.whl size=26798 sha256=2a6e2362a8c42d92b91d0a83ca675db32caa8715ca8dd50b4e86a0c5beb65db7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpnkezg8/wheels/28/93/20/67dca95c2aaa13466b4900ba79a7bab66022e50ce44f8a438d
Successfully built hpsklearn


In [ ]:
! pip install scikit-learn==0.21
! pip install pandas==0.24.2
! pip install pandas_ml==0.5.0

  Using cached https://files.pythonhosted.org/packages/ae/72/6d90debfcb9ea74ec00927fa7ed0204dcc560b1f9ffcd8b239daa7fd106d/pandas_ml-0.6.1-py3-none-any.whl


In [ ]:
! pip uninstall -y pandas_ml
! pip install pandas_ml

Uninstalling pandas-ml-0.5.0:
  Successfully uninstalled pandas-ml-0.5.0


In [ ]:
import model
import generator
import classes
import utils

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os

import tensorflow as tf
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import TensorBoard
from callbacks import ConfusionMatrixCallback
from model import speech_model, prepare_model_settings
from generator import AudioProcessor, prepare_words_list
from classes import get_classes
from utils import data_gen

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
# args = parser.parse_args()
args = {}
args['data_dirs'] = train_dir
args['sample_rate'] = 16000
args['batch_size'] = 64
args['output_representation'] = 'raw'
# parser.print_help()
print('input args: ', args)

input args:  {'data_dirs': PosixPath('/content/data/train'), 'sample_rate': 16000, 'batch_size': 64, 'output_representation': 'raw'}


In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 53kB/s 


In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras import backend as K
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [ ]:

p = args['data_dirs']
print("p = ", p)
data_dirs = [str(x) for x in p.iterdir() if x.is_dir()]
# data_dirs=(args['data_dirs'])
print("data_dirs = ", data_dirs)
output_representation = args['output_representation']
sample_rate = args['sample_rate']
batch_size = args['batch_size']
classes = get_classes(wanted_only=True)
print("classes = ", classes)
model_settings = prepare_model_settings(
    label_count=len(prepare_words_list(classes)),
    sample_rate=sample_rate,
    clip_duration_ms=1000,
    window_size_ms=30.0,
    window_stride_ms=10.0,
    dct_coefficient_count=80,
    num_log_mel_features=60,
    output_representation=output_representation)

print(model_settings)

p =  /content/data/train
data_dirs =  ['/content/data/train/cat', '/content/data/train/no', '/content/data/train/house', '/content/data/train/one', '/content/data/train/two', '/content/data/train/off', '/content/data/train/three', '/content/data/train/up', '/content/data/train/seven', '/content/data/train/stop', '/content/data/train/left', '/content/data/train/happy', '/content/data/train/go', '/content/data/train/down', '/content/data/train/marvin', '/content/data/train/bird', '/content/data/train/wow', '/content/data/train/nine', '/content/data/train/bed', '/content/data/train/zero', '/content/data/train/five', '/content/data/train/right', '/content/data/train/_background_noise_', '/content/data/train/six', '/content/data/train/yes', '/content/data/train/sheila', '/content/data/train/eight', '/content/data/train/dog', '/content/data/train/on', '/content/data/train/four', '/content/data/train/tree']
classes =  ['stop', 'down', 'off', 'right', 'up', 'go', 'on', 'yes', 'left', 'no']
{'d

In [ ]:
tf.compat.v1.disable_eager_execution()
ap = AudioProcessor(
    data_dirs=data_dirs,
    wanted_words=classes,
    silence_percentage=13.0,
    unknown_percentage=60.0,
    validation_percentage=10.0,
    testing_percentage=0.0,
    model_settings=model_settings,
    output_representation=output_representation)
train_gen = data_gen(ap, sess, batch_size=batch_size, mode='training')
val_gen = data_gen(ap, sess, batch_size=batch_size, mode='validation')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



shape of self.spectrogram_ =  Tensor("Shape:0", shape=(3,), dtype=int32)
prepare_processing_graph completed


In [ ]:
model = speech_model(
    'conv_1d_time_stacked',
    model_settings['fingerprint_size']
    if output_representation != 'raw' else model_settings['desired_samples'],
    # noqa
    num_classes=model_settings['label_count'],
    **model_settings)

In [ ]:
# embed()
checkpoints_path = os.path.join(checkpt_dir, 'conv_1d_time_stacked_model')
if not os.path.exists(checkpoints_path):
  os.makedirs(checkpoints_path)

In [ ]:
callbacks = [
      ConfusionMatrixCallback(
          val_gen,
          ap.set_size('validation') // batch_size,
          wanted_words=prepare_words_list(get_classes(wanted_only=True)),
          all_words=prepare_words_list(classes),
          label2int=ap.word_to_index),
      ReduceLROnPlateau(
          monitor='val_categorical_accuracy',
          mode='max',
          factor=0.5,
          patience=4,
          verbose=1,
          min_lr=1e-5),
      TensorBoard(log_dir='logs'),
      ModelCheckpoint(
          os.path.join(checkpoints_path,
                       'ep-{epoch:03d}-vl.hdf5'),
          save_best_only=True,
          monitor='val_loss',
          mode='max')
  ]

In [ ]:
model.fit(
    train_gen,
    steps_per_epoch=ap.set_size('training') // batch_size,
    epochs=50,
    verbose=1,
    callbacks=callbacks)

Epoch 1/50
525/570 [==========================>...] - ETA: 5s - batch: 262.0000 - size: 64.0000 - loss: 0.0394 - categorical_accuracy: 0.9946
[Ep:062: training-mode]
570/570 [==============================] - ETA: 0s - batch: 284.5000 - size: 64.0000 - loss: 0.0398 - categorical_accuracy: 0.9945
[Ep:064: validation-mode]
570/570 [==============================] - 82s 144ms/step - batch: 284.5000 - size: 64.0000 - loss: 0.0398 - categorical_accuracy: 0.9945
Epoch 2/50
525/570 [==========================>...] - ETA: 5s - batch: 262.0000 - size: 64.0000 - loss: 0.0391 - categorical_accuracy: 0.9951
[Ep:063: training-mode]
570/570 [==============================] - ETA: 0s - batch: 284.5000 - size: 64.0000 - loss: 0.0390 - categorical_accuracy: 0.9951
[Ep:065: validation-mode]
570/570 [==============================] - 81s 142ms/step - batch: 284.5000 - size: 64.0000 - loss: 0.0390 - categorical_accuracy: 0.9951
Epoch 3/50
525/570 [==========================>...] - ETA: 5s - batch: 262.000

In [ ]:
model_dir = Path(base_dir,'speech_commands.hdf5')
model.save(model_dir)

In [ ]:
eval_res = model.evaluate_generator(val_gen,
                                      ap.set_size('validation') // batch_size)


[Ep:063: validation-mode]


In [ ]:
print(eval_res)

[0.2604759057470854, 0.9413496]
